# Indexed cellular automation

- <https://adventofcode.com/2021/day/20>

This is another cellular automaton problem, one where [`scipy.signal.convolve2d()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.convolve2d.html) really shines!

For each 3x3 set of cells in the input image (matrix), you basically turn the pixels into a binary number, the value of which is an index into the 'algorithm' array of the puzzle input. Since `convolve2d` essentially uses the input kernel as a set of weights to sum for each pixel in the image that is on, we only have to give the kernel the right weights for each binary digit in that number; $2^0$ for the first, $2^1$ for the second, etc.:

$$\begin{bmatrix}2^0 & 2^1 & 2^2\\2^3 & 2^4 & 2^5\\2^6 & 2^7 & 2^8\end{bmatrix}$$

You can then apply the output of `convolve2d()` directly to the `algorithm` array to get the next iteration of the image.

There is one small catch: depending on the values for indices `0` and `511` in the 'algorithm', there are actually 3 different possibilities for what the `fillvalue` should be for the `convolve2d()` call. The `fillvalue` is what convolve2d will use for those cells at the edges of the image, because numpy arrays are not infinite.

The first possibility is that the edge is forever 'off'; all image elements outside of the matrix are set to `0`, always. This is the case if `algorithm[0]` is `0` itself, so any random 3x3 set of pixels outside from the 'visible' pixels will always have 9 zeros, the binary value for those pixels will always be zero, and the replacement value in the output is zero. Simple.

But there are two other scenarios possible:

- `algorithm[0]` is `1` and `algorithm[511]` is `0`. Now, the whole infinite image is going to _flip between states_. The first step sees all empty 3x3 sets of pixels turn to `1` in the output, resulting in an infinity of `111111111` or `511` values, so the next step then turns them all off again by outputing `0`.

- `algorithm[0]` is `1` and `algorithm[511]` is also `1`. This means that on the first step the infinite image turns black (outputs `1` for the blank areas), and then it stays there.

The second scenario is only theoretically possible, but no Advent of Code puzzle input will ever contain it because then you'd have to have a way to answer 'infinite' to the question 'how many pixels are lit'. Instead, the puzzle _carefully but deliberately_ asks for the number of lit pixels after an even number of steps, when both the first (test) scenario and the second (flip-flopping) scenario would both result in a finite number of lit pixels.

In any case, we don't have to track an infinite state, we only have to track what value to give `fillvalue` each step. I created an `edge_fill` property that returns an infinite iterator yielding the right values; either an infinite repetition of `False` (test scenario), an infinite sequence of alternating `False` and `True` values (flip-flopping), or a sequence of a single `False` value followed by an infinity of `True` values (the 3rd, never encountered, scenario).


In [1]:
from __future__ import annotations
from dataclasses import dataclass
from itertools import chain, cycle, islice, repeat
from typing import Final, Iterator

import numpy as np
from scipy.signal import convolve2d


KERNEL: Final[np.array] = 2 ** np.arange(9, dtype=np.uint16).reshape(3, 3)


@dataclass
class ImageEnhancer:
    image: np.array[bool]
    algorithm: np.array[bool]

    @classmethod
    def from_scanner(cls, lines: str) -> ImageEnhancer:
        algo, _, image = lines.partition("\n\n")

        return cls(
            np.array([[c == "#" for c in line] for line in image.splitlines()]),
            np.array([c == "#" for c in algo]),
        )

    @property
    def edge_fill(self) -> Iterator[bool]:
        blank, full = self.algorithm[[0, -1]]
        if not blank:
            return repeat(False)
        if not full:
            return cycle((False, True))
        return chain((False,), repeat(True))

    def enhance(self, steps: int) -> np.array[bool]:
        img, algo = self.image, self.algorithm
        for edge in islice(self.edge_fill, steps):
            img = algo[convolve2d(img, KERNEL, fillvalue=edge)]
        return img


test_image = ImageEnhancer.from_scanner(
    """\
..#.#..#####.#.#.#.###.##.....###.##.#..###.####..#####..#....#..#..##..##\
#..######.###...####..#..#####..##..#.#####...##.#.#..#.##..#.#......#.###\
.######.###.####...#.##.##..#..#..#####.....#.#....###..#.##......#.....#.\
.#..#..##..#...##.######.####.####.#.#...#.......#..#.#.#...####.##.#.....\
.#..#...##.#.##..#...##.#.##..###.#......#.#.......#.#.#.####.###.##...#..\
...####.#..#..#.##.#....##..#.####....##...##..#...#......#.#.......#.....\
..##..####..#...#.#.#...##..#.#..###..#####........#..####......#..#

#..#.\n#....\n##..#\n..#..\n..###
"""
)

assert test_image.enhance(2).sum() == 35


In [2]:
import aocd

scanned_image = ImageEnhancer.from_scanner(aocd.get_data(day=20, year=2021))
print("Part 1:", scanned_image.enhance(2).sum())


Part 1: 5275


# Part 2

Part two is just the same as part one, just with more steps. We already found an efficient way of handling this, so no changes were necessary.


In [3]:
assert test_image.enhance(50).sum() == 3351


In [4]:
print("Part 2:", scanned_image.enhance(50).sum())


Part 2: 16482
